In [4]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf   
from sklearn.preprocessing import scale
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

In [2]:
tf.__version__

'2.0.1'

In [3]:
tf.test.is_gpu_available()

True

In [16]:
datas[0]

'test_transfer_gps/indoor/09/92.txt'

In [19]:
datas = glob.glob('test_transfer_gps/*/*/*.txt')
f = np.loadtxt(datas[0],delimiter=",")

array([[-14449808.30148969,  14196109.80994526,  17796159.32796757,
         21359219.70882853],
       [ -3948999.40915668,  18009370.84073456,  18903995.43685982,
         20942551.66359347],
       [-13094086.18535668,  -8136146.14246707,  21579381.07911287,
         25419861.48750373],
       [  9865483.03759483,  24394321.73617796,  -4406369.70133495,
         24160993.09489245],
       [-12825233.25972143,  22919493.82973369,  -3694444.95697328,
         21291745.12639669],
       [ -1188319.76545269,  24356378.41594348, -10376427.94633646,
         23302528.39422349],
       [-18234595.56102006,   8935818.52759861, -16675956.08928304,
         24703619.23735278],
       [-18960448.41531217,  14817216.0610646 , -11496672.71748914,
         23403325.21444231],
       [ 16820963.3992603 ,  20081677.50927563,   4925663.64476027,
         24948201.31468994],
       [  8842986.9323955 ,  12815506.22219114,  21464810.67701118,
         23712243.64450322]])

In [4]:
dataset = pd.read_csv("trainingData.csv",header = 0)

features = np.asarray(dataset.iloc[:,0:520])
features[features == 100] = -110
features = (features - features.mean()) / features.var()

labels = np.asarray(dataset["BUILDINGID"].map(str) + dataset["FLOOR"].map(str))
labels = np.asarray(pd.get_dummies(labels))

In [5]:
train_val_split = np.random.rand(len(features)) < 0.70
train_x = features[train_val_split]
train_y = labels[train_val_split]
val_x = features[~train_val_split]
val_y = labels[~train_val_split]

In [6]:
test_dataset = pd.read_csv("validationData.csv",header = 0)

test_features = np.asarray(test_dataset.iloc[:,0:520])
test_features[test_features == 100] = -110
test_features = (test_features - test_features.mean()) / test_features.var()

test_labels = np.asarray(test_dataset["BUILDINGID"].map(str) + test_dataset["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [7]:
train_x.shape

(13999, 520)

In [8]:
test_features.shape

(1111, 520)

In [9]:
labels.shape[1]

13

In [10]:
latent_dim = 64 

class Autoencoder(Model):
  def __init__(self, encoding_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(128, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(520, activation='sigmoid')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim)

In [11]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [12]:
autoencoder.fit(train_x, train_x,
                epochs=1000,
                shuffle=True,
                validation_data=(val_x, val_x))

Train on 13999 samples, validate on 5938 samples
Epoch 1/1000
13999/13999 [==============================] - 5s 379us/sample - loss: 0.0321 - val_loss: 0.0258
Epoch 2/1000
13999/13999 [==============================] - 4s 292us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 3/1000
13999/13999 [==============================] - 4s 292us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 4/1000
13999/13999 [==============================] - 4s 298us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 5/1000
13999/13999 [==============================] - 4s 290us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 6/1000
13999/13999 [==============================] - 4s 296us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 7/1000
13999/13999 [==============================] - 4s 300us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 8/1000
13999/13999 [==============================] - 4s 279us/sample - loss: 0.0259 - val_loss: 0.0258
Epoch 9/1000
13999/13999 [==============================] - 4s 292us/sa

In [13]:
DNN = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(labels.shape[1], activation='softmax'),
    ])

In [14]:
train_encoded = np.array(autoencoder.encoder(train_x))
val_encoded = np.array(autoencoder.encoder(val_x))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [15]:
DNN.compile(optimizer='adam', loss=losses.CategoricalCrossentropy())

In [16]:
DNN.fit(train_encoded, train_y,
                epochs=1000,
                shuffle=True,
                validation_data=(val_encoded, val_y))

Train on 13999 samples, validate on 5938 samples
Epoch 1/1000
13999/13999 [==============================] - 4s 257us/sample - loss: 0.6038 - val_loss: 0.4183
Epoch 2/1000
13999/13999 [==============================] - 3s 212us/sample - loss: 0.2611 - val_loss: 0.2185
Epoch 3/1000
13999/13999 [==============================] - 3s 217us/sample - loss: 0.1956 - val_loss: 0.1939
Epoch 4/1000
13999/13999 [==============================] - 3s 216us/sample - loss: 0.1603 - val_loss: 0.1579
Epoch 5/1000
13999/13999 [==============================] - 3s 209us/sample - loss: 0.1377 - val_loss: 0.1582
Epoch 6/1000
13999/13999 [==============================] - 3s 208us/sample - loss: 0.1223 - val_loss: 0.1465
Epoch 7/1000
13999/13999 [==============================] - 3s 212us/sample - loss: 0.1055 - val_loss: 0.1203
Epoch 8/1000
13999/13999 [==============================] - 3s 207us/sample - loss: 0.0995 - val_loss: 0.1310
Epoch 9/1000
13999/13999 [==============================] - 3s 212us/sa

In [17]:
test_encoder = np.array(autoencoder.encoder(test_features))
y_ = DNN(test_encoder)

In [18]:
np.argmax(np.array(y_[2]))

12